### Contoh membuat GeoSeries sederhana

In [1]:
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon

# Membuat GeoSeries dari objek geometris
geo_series = gpd.GeoSeries([
    Point(1, 1),  # Titik
    LineString([(0, 0), (1, 1), (2, 2)]),  # Garis
    Polygon([(0, 0), (1, 0), (1, 1), (0, 1)])  # Poligon
])

geo_series.head()

0                            POINT (1 1)
1             LINESTRING (0 0, 1 1, 2 2)
2    POLYGON ((0 0, 1 0, 1 1, 0 1, 0 0))
dtype: geometry

### Contoh membuat GeoDataFrame

In [2]:
# import geopandas as gpd

# Membuat GeoDataFrame dari GeoSeries
gdf = gpd.GeoDataFrame({
    'name': ['Point A', 'Line B', 'Polygon C'],
    'geometry': geo_series # variable geo_series disini diambil dari source code sub judul diatas (GeoSeries)
})

gdf.head()

,name,geometry
0,Point A,POINT (1 1)
1,Line B,"LINESTRING (0 0, 1 1, 2 2)"
2,Polygon C,"POLYGON ((0 0, 1 0, 1 1, 0 1, 0 0))"


### Menyeleksi Data dalam GeoDataFrame

In [3]:
# Seleksi baris berdasarkan kondisi
# variable gdf disini diambil dari source code sub judul diatas (GeoDataFrame)
# 'name' adalah penamaan column yang akan difilter
selected = gdf[gdf['name'] == 'Point A']
selected.head()

,name,geometry
0,Point A,POINT (1 1)


### Operasi Geometri Umum

In [4]:
# variable gdf disini diambil dari source code sub judul diatas (GeoDataFrame)
# sebuah variable GeoDataFrame ketika mengakses gdf['contoh'] maka akan menambah column 'contoh' baru ketika belum ada penamaan column tersebut

# Menghitung centroid dari setiap geometri
gdf['centroid'] = gdf.geometry.centroid

# Menghasilkan convex hull dari setiap geometri
gdf['convex_hull'] = gdf.geometry.convex_hull

gdf[['name', 'geometry', 'centroid', 'convex_hull']]

,name,geometry,centroid,convex_hull
0,Point A,POINT (1 1),POINT (1 1),POINT (1 1)
1,Line B,"LINESTRING (0 0, 1 1, 2 2)",POINT (1 1),"LINESTRING (0 0, 2 2)"
2,Polygon C,"POLYGON ((0 0, 1 0, 1 1, 0 1, 0 0))",POINT (0.5 0.5),"POLYGON ((0 0, 0 1, 1 1, 1 0, 0 0))"


### Penggabungan dan Penyaringan Data

In [6]:
# import geopandas as gpd
# from shapely.geometry import Point, Polygon

# Example GeoDataFrame 1: Points (earthquake locations)
data1 = {
    'id': [1, 2, 3],
    'place': ['Tokyo', 'San Francisco', 'Jakarta'],
    'latitude': [35.6762, 37.7749, -6.2088],
    'longitude': [139.6503, -122.4194, 106.8456]
}
geometry1 = [Point(xy) for xy in zip(data1['longitude'], data1['latitude'])]
gdf1 = gpd.GeoDataFrame(data1, geometry=geometry1)

# Example GeoDataFrame 2: Polygons (country borders)
polygons = [
    Polygon([(-125, 35), (-115, 35), (-115, 40), (-125, 40), (-125, 35)]),  # USA
    Polygon([(130, 30), (140, 30), (140, 40), (130, 40), (130, 30)]),       # Japan
    Polygon([(100, -10), (110, -10), (110, 0), (100, 0), (100, -10)])       # Indonesia
]
data2 = {'id': [1, 2, 3], 'country': ['USA', 'Japan', 'Indonesia']}
gdf2 = gpd.GeoDataFrame(data2, geometry=polygons)

# Perform spatial join (where points intersect polygons)
joined_gdf = gpd.sjoin(gdf1, gdf2, how="inner", predicate='intersects')

# Display the result of the join
joined_gdf.head()

,id_left,place,latitude,longitude,geometry,index_right,id_right,country
0,1,Tokyo,35.6762,139.6503,POINT (139.6503 35.6762),1,2,Japan
1,2,San Francisco,37.7749,-122.4194,POINT (-122.4194 37.7749),0,1,USA
2,3,Jakarta,-6.2088,106.8456,POINT (106.8456 -6.2088),2,3,Indonesia
